# Reinforcement learning from (human) feedback

библиотека TRL - https://github.com/lvwerra/trl 

In [1]:
%pip install transformers trl wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### TRL

Схематично обучение в TRL устроено вот так:

1) Rollout - это просто генерация текстов, используя исходную языковую модель. Чтобы тексты были релеватными можно взять определенные промты и генерировать их продолжения.

2) Evaluation - это оценивание сгенерированных текстов, в нашем случае мы будет это делать с помощью предобученного классификатора тональности 

3) Optimization - это само обучение. Обратите внимание, что на этом шаге есть active_model и reference_model. Изначально это одна и та же модель, но в процессе обучения мы изменяем active_model так, чтобы она не сильно уходила от изначальное модели (reference_model)

In [2]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [3]:
# тут задаются гиперпараметры
config = PPOConfig(
    model_name="ai-forever/rugpt3small_based_on_gpt2",
    learning_rate=1.41e-5,
    log_with=None,
    mini_batch_size=16
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/labeled.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment', 'toxic'],
        num_rows: 14412
    })
})

Для генерирования текстов тут используется toxic датасет. От каждого текста отрезается случайно какой-то кусочек и модель должна его продолжить.

In [7]:
def build_dataset(config, dataset=dataset['train'], input_min_text_length=2, input_max_text_length=8):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = dataset
    ds = ds.rename_columns({"comment": "review"})
    
    # еще есть фильтрация по длине
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 2000, batched=False)
    
    # длина кусочка определяется случайно
    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [8]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# active_model
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
# reference_model (обратите внимание что это одна и так же модель изначально)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [11]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

Для оценки тональности используется пайплайн из huggingface и предобученная модель

In [12]:
sentiment_pipe = pipeline("sentiment-analysis", model="cointegrated/rubert-tiny-sentiment-balanced", device=device)

Для каждого текста она выдает скор негативности и положительности. Для обучения дальше будет использоваться скор положительности

In [13]:
text = "этот фильм всрат!!"
sentiment_pipe(text, **sent_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'negative', 'score': -0.9139207601547241},
  {'label': 'neutral', 'score': 0.46272024512290955},
  {'label': 'positive', 'score': 0.11588776111602783}]]

In [14]:
text = "этот фильм отличный!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -1.3060051202774048},
  {'label': 'neutral', 'score': -1.5412918329238892},
  {'label': 'positive', 'score': 2.711308240890503}]]

Еще одно место, где можно настраивать параметры. Эти параметры мы разбирали на семинаре по GPT они отвечают за генерацию.

In [15]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, 
              "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

Само обучение. Для небольших моделей оно даже не очень долгое

In [16]:
output_min_length = 8
output_max_length = 32
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), 
                         total=dataset.num_rows//ppo_trainer.dataloader.batch_sampler.batch_size):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[2]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

  0%|          | 0/13 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [17]:
# pd.set_option('display.max_rows', 3000)
# pd.set_option('display.max_colwidth', 5000)

Можно посмотреть на результаты, сгенериров продолжения старой и новой модель

In [20]:
#### get a batch from the dataset
bs = 32
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[2]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[2]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [21]:
pd.DataFrame.from_dict(game_data)

,query,response (before),response (after),rewards (before),rewards (after)
0,Пфф,"ф, все ж зацепил!\nБог велел, блять!\nну он то...","ф, нет Спасибо). Обработана память - правда.\n...",-1.854557,3.233626
1,В этих случаях ищут максимальные,"возможности, т.к. в изоляторах полнейший кари...",шансы опережающего развития событий).\n\nВ ит...,-0.664224,-0.225002
2,Умеет - и,пальцем не тронет.\nЛегко и непринужд,обязательно умеет заставить) \n Люблю зимы и ...,-1.579653,3.815419
3,"Во, у нас такие же","от восточнейшего колеи, знаете",забронированные окна!\nВот тут,-1.138388,-1.488920
4,вот ты мне только,"честно, ты -- одно целое с непобедимой верой ...",что подарил такую замечательную девушку....\n...,-1.407189,3.909848
5,СЕГОДНЯ,в Афинах Вас приветствуют флорентийцы и венец...,). В этом году дата знаменательна максимально...,-1.952533,-1.497386
6,Ну а что,с остальными бедами? С каждым.\nа с,", единственное занятие, которое тебя развлечет...",-2.951481,-2.088066
7,Вы правы. Но:,я никоим образом не изучала эти тайны. Способ...,"-) Я не люблю, когда меня хотят посвящать нас...",-1.728553,0.233797
8,"Ну,",веселая может быть я? – хмыкнула реплика Ильи...,"ты ничего не хочешь мне сказать, милый?\n\nВа...",-3.455989,-3.186114
9,"Скримеры рак, форся","ки (роза,папахиалям). Алкалоид кон",", марада кого?)\n\n\n\n\n.. о чём ты",-1.127897,0.752032


Видно, что модель теперь генерирует больше положительных текстов